In [ ]:
from tests.test_attention import flash_backward_results, _attention_and_lse, _make_attn_inputs
from cs336_systems.flashattention_autograd_function import FlashAttentionAutogradFunctionTriton as FlashAttention2

In [ ]:
def test_timing_flash_forward_backward():
    n_heads = 16
    d_head = 64
    sequence_length = 16384
    q, k, v = torch.randn(
        3, n_heads, sequence_length, d_head, device='cuda', dtype=torch.bfloat16, requires_grad=True
    )
    
    flash = torch.compile(FlashAttention2.apply)
    
    def flash_forward_backward():
        o = flash(q, k, v, True)
        loss = o.sum()
        loss.backward()
    
    results = triton.testing.do_bench(flash_forward_backward, rep=10000, warmup=1000)
    print(results)

In [ ]:
grad_out = do
Q, K, V, L, O = o.grad_fn.saved_tensors
batch_size, _, D = Q.shape
DD = torch.sum(O * grad_out, axis=-1)
S = einsum(Q, K, "... q d, ... k d -> ... q k") / D ** 0.5
P = torch.exp(S - L[:, :, None])
dV = einsum(P, grad_out, "... q k, ... q d -> ... k d")
dP = einsum(grad_out, V, "... q d, ... k d -> ... q k")
dS = P * (dP - DD[:, :, None])
dQ = einsum(dS, K, "... q k, ... k d -> ... q d") / D ** 0.5
dK = einsum(dS, Q, "... q k, ... q d -> ... k d") / D ** 0.5

In [ ]:
dq_e